In [5]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [41]:
data = pd.read_table('data/dev.tsv')
test = pd.read_table('data/eval.tsv')

In [42]:
df = data.copy()
eval = test.copy()

In [43]:
from scipy import stats
def happy_sad(x):
    if x>df['valence'].mean():
        return 'happy'
    else:
        return 'sad'
df['boringness'] = df['loudness'] + df['tempo'] + (df['energy']*100) + (df['danceability']*100)
df['valence_happy_sad'] = df['valence'].apply(lambda x: happy_sad(x))
df['loudness_plus_60'] = df['loudness'].apply(lambda x: x+60)
df['loudness_pos'] = df['loudness'].apply(lambda x: -1*x)
df['loudness_pos'] = np.sqrt(df['loudness_pos'])
df['boringness_plus_60'] = df['boringness'].apply(lambda x: x+60)
df['duration_ms_box_cox_trans'] = stats.boxcox(df['duration_ms'])[0]
df['acousticness_sqrt_trans'] = np.sqrt(df['acousticness'])
df['liveness_sqrt_trans'] = np.sqrt(df['liveness'])
df['popularity_sqrt_trans'] = np.sqrt(df['popularity'])
df['speechiness_sqrt_trans'] = np.sqrt(df['speechiness'])

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [44]:
df = df.fillna(value=0)

In [52]:
col = [
    'valence',
        'year',
         'acousticness',
        # 'artists',
        'danceability',
        'duration_ms',
        'energy',
        'explicit',
        # 'id',
        'instrumentalness',
        'key',
       'liveness',
         'loudness',
        'popularity',
         'speechiness',
        'tempo',
         'mode',
        'loudness_plus_60',
        'loudness_pos',
         'boringness',
         # 'valence_happy_sad',
        'boringness_plus_60',
        'duration_ms_box_cox_trans',
       'acousticness_sqrt_trans',
        'liveness_sqrt_trans',
        'popularity_sqrt_trans',
        'speechiness_sqrt_trans'
        ]
X = df[col]
y = df['mode']

In [16]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
valence,136522.0,0.528540,0.263282,0.0,0.3170,0.540000,0.74700,1.000
year,136522.0,1976.796048,25.921025,1921.0,1956.0000,1977.000000,1999.00000,2020.000
acousticness,136522.0,0.502159,0.376037,0.0,0.1020,0.516000,0.89300,0.996
danceability,136522.0,0.537497,0.175972,0.0,0.4150,0.548000,0.66800,0.988
duration_ms,136522.0,231061.453041,127800.577241,5991.0,169933.0000,207493.000000,262267.00000,5403500.000
energy,136522.0,0.482298,0.267538,0.0,0.2550,0.471000,0.70300,1.000
explicit,136522.0,0.084411,0.278005,0.0,0.0000,0.000000,0.00000,1.000
instrumentalness,136522.0,0.167172,0.313448,0.0,0.0000,0.000218,0.10300,1.000
key,136522.0,5.193200,3.513427,0.0,2.0000,5.000000,8.00000,11.000
liveness,136522.0,0.205708,0.174404,0.0,0.0989,0.136000,0.26100,1.000


In [21]:
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, y)

# Summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)

features = fit.transform(X)

[5.382e+00 5.102e+01 8.597e+01 1.714e+01 1.984e+07 3.159e+01 8.027e+02
 1.091e+02 4.450e+03 1.441e-01 7.567e+00 1.841e+03 1.196e+02 1.697e+02
 1.224e+03]


In [22]:
print(features)

[[1.209e+05 4.000e+00 6.100e+01 2.905e+02]
 [3.227e+05 5.000e+00 0.000e+00 1.337e+02]
 [2.825e+05 0.000e+00 2.900e+01 2.085e+02]
 ...
 [1.772e+05 7.000e+00 0.000e+00 1.866e+02]
 [1.617e+05 5.000e+00 2.100e+01 2.285e+02]
 [2.975e+05 7.000e+00 3.500e+01 2.889e+02]]


In [48]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
rfe = RFE(model, 7)
fit = rfe.fit(X, y)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_features_to_select=7 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATION

Num Features: 7
Selected Features: [ True False  True  True  True  True False False False False False False
  True False  True]
Feature Ranking: [1 8 1 1 1 1 3 2 9 6 4 7 1 5 1]


In [53]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0)
ridge.fit(X,y)
def pretty_print_coefs(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)
print ("Ridge model:", pretty_print_coefs(ridge.coef_))

Ridge model: 0.0 * X0 + 0.0 * X1 + -0.0 * X2 + -0.0 * X3 + 0.0 * X4 + 0.0 * X5 + -0.0 * X6 + -0.0 * X7 + -0.0 * X8 + 0.0 * X9 + 0.0 * X10 + -0.0 * X11 + 0.0 * X12 + 0.0 * X13 + 1.0 * X14 + 0.0 * X15 + 0.0 * X16 + -0.0 * X17 + -0.0 * X18 + -0.0 * X19 + 0.0 * X20 + -0.0 * X21 + 0.0 * X22 + -0.0 * X23


X0 X5 X9 X10 X12 X13 X18 X21
valence energy liveness loudness speechiness tempo popularity_sqrt_trans

In [40]:
X.columns

Index(['valence', 'year', 'acousticness', 'danceability', 'duration_ms',
       'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'popularity', 'speechiness', 'tempo', 'boringness', 'loudness_plus_60',
       'boringness_plus_60', 'duration_ms_box_cox_trans',
       'acousticness_sqrt_trans', 'liveness_sqrt_trans', 'loudness_pos',
       'popularity_sqrt_trans', 'speechiness_sqrt_trans'],
      dtype='object')